In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def load_midi_data(data_dir):
    import midi.utils as utils

    midi_data = utils.midiread(data_dir, dt=0.5)

    return midi_data.piano_roll

In [3]:
midi = load_midi_data('sample1.mid')
train_x = midi[:120].astype(np.float32)
n_x = train_x.shape[1]
n_hidden = [500]
n_z = 2
n_y = n_x

frames  = train_x.shape[0]
n_batch = 6
seq_length = frames / n_batch

split_x = np.vsplit(train_x, n_batch)
n_input = 20
n_epochs = 500

In [5]:
from tensorflow.models.rnn import rnn, rnn_cell

In [6]:
class VARE(object):
    
    def __init__(self, batch_size, n_z, n_hidden, n_input, train_x, optimizer = tf.train.AdamOptimizer()):
        self.learning_rate = 0.001
        self.batch_size = batch_size
        self.n_hidden = n_hidden
        self.n_input = n_input
        self.train_x = train_x
        
        self.x = tf.placeholder(tf.float32, [batch_size, n_input])
        
        basic_cell = rnn_cell.BasicRNNCell(self.n_hidden[0])
        self._initial_state = basic_cell.zero_state(batch_size, tf.float32)
        state = self._initial_state
        
        with tf.variable_scope("RNN") as scope:
            for time_step in range(n_input):
                # x_in_t = tf.Variable(self.x[time_step].reshape((1, self.x.shape[1])))
                x_in_t = tf.reshape(tf.slice(self.x, [0, 0], [1, n_input]), (1, n_input))
                # x_in_t = self.x[time_step].reshape((1, self.x.shape[1]))
                if time_step > 0:
                    scope.reuse_variables()
                cell_output, state = basic_cell(x_in_t, state)
        
        network_weights = self._initialize_weights()
        self.weights = network_weights
        
        q_mean = tf.add(tf.matmul(state, self.weights['recog_mean']), self.weights['recog_mean_b'])
        log_sigma_sq = 0.5 * tf.add(tf.matmul(state, self.weights['recog_log_sigma']), self.weights['recog_log_sigma_b'])
        
        eps = tf.random_normal(tf.shape(log_sigma_sq), 0, 1, dtype = tf.float32)

        self.z = tf.add(q_mean, tf.mul(tf.exp(log_sigma_sq), eps))
        
        h0 = tf.nn.tanh(tf.add(tf.matmul(self.z, self.weights['h0']), self.weights['h0_b']))
        out= tf.add(tf.matmul(h0, self.weights['out']), self.weights['out_b'])
        
        x_0 = tf.nn.tanh(out)
        
        output[0] = x_0.data
        
        rec_loss = tf.nn.sigmoid_cross_entropy_with_logits(out, tf.Variable(self.x[0].reshape((1, self.x.shape[1])).astype(np.int32)))
        
        x_t = x_0
        
        basic_cell2 = rnn_cell.BasicRNNCell(self.n_hidden[0])
        
        with tf.variable_scope("h0") as scope:
            for i in range(1, self.x.shape[0]):
                if i > 0:
                    scope.reuse_variables()
                (cell_output, state) = basic_cell(x_t, state)
                x_t_1 = tf.add(tf.matmul(state, self.weights['out']), self.weights['out_b'])
                rec_loss += tf.nn.sigmoid_cross_entropy_with_logits(out, Variable(self.x[i].reshape((1,self.x.shape[1])).astype(np.int32)))
                x_t = output_t = tf.nn.tanh(x_t_1)
                output[i]  = output_t.data
        
        KLD = -0.0005 * tf.reduce_sum(1 + log_sigma_sq 
                                           - tf.square(q_mean) 
                                           - tf.exp(log_sigma_sq), 1)
        
        self.cost = tf.reduce_mean(rec_loss + KLD)
        
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
        #init = tf.initialize_all_variables()
        #self.sess = tf.Session()
        #self.sess.run(init)
                
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['recog_mean'] = tf.Variable(tf.zeros([self.n_hidden[-1], 2], dtype=tf.float32))
        all_weights['recog_mean_b'] = tf.Variable(tf.zeros([n_z], dtype=tf.float32))
        all_weights['recog_log_sigma'] = tf.Variable(tf.zeros([self.n_hidden[0], n_z], dtype=tf.float32))
        all_weights['recog_log_sigma_b'] = tf.Variable(tf.zeros([n_z], dtype=tf.float32))
        all_weights['h0'] = tf.Variable(tf.zeros([n_z, n_hidden[0]], dtype=tf.float32))
        all_weights['h0_b'] = tf.Variable(tf.zeros([self.n_hidden[0]], dtype=tf.float32))
        all_weights['out'] = tf.Variable(tf.zeros([self.n_hidden[-1], self.train_x.shape[1]], dtype=tf.float32))
        all_weights['out_b'] = tf.Variable(tf.zeros([self.train_x.shape[1]], dtype=tf.float32))
        all_weights['gen_in_h'] = tf.Variable(tf.zeros([self.train_x.shape[1], self.train_x.shape[1]], dtype=tf.float32))
        all_weights['gen_in_h_b'] = tf.Variable(tf.zeros([self.train_x.shape[1]], dtype=tf.float32))
        all_weights['gen_h_h'] = tf.Variable(tf.zeros([self.n_hidden[-1], self.train_x.shape[1]], dtype=tf.float32))
        all_weights['gen_h_h_b'] = tf.Variable(tf.zeros([self.train_x.shape[1]], dtype=tf.float32))
        return all_weights
    
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict={self.x: X})
        return cost
    

In [7]:
display_step = 1

model = VARE(n_batch, n_z, n_hidden, n_input, train_x)

for epoch in xrange(1, n_epochs + 1):
    print('epoch', epoch)
    t1 = time.time()
    total_loss = 0.0
    outputs = np.zeros(train_x.shape, dtype=np.float32)
    for i in xrange(n_batch):
        x_batch = split_x[i]
        print x_batch
        cost = model.partial_fit(x_batch)
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch + 1), \
                "cost=", "{:.9f}".format(cost)


AttributeError: 'Tensor' object has no attribute 'data'

In [11]:
n_hidden[-1]

500